<a href="https://colab.research.google.com/github/quockhanhdao/IASLab/blob/MNIST_MLP/MNIST_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neptune-client neptune-tensorflow-keras tensorflow

## Upload file

In [ ]:
from google.colab import files
upload = files.upload()

## Intro

In [ ]:
import tensorflow as tf
from keras.datasets import mnist
from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical
import numpy as np

https://app.neptune.ai/khanh.daoquoc/test-proj/e/TES-14
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## Track model

In [ ]:
import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

run = neptune.init(
    project="khanh.daoquoc/test-proj",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3Mjg4YTFkMy05ODlmLTQ3OWUtOWJhZS0yYjk5YmJlMmQ2NTMifQ==",
)

params = {"layer_1": 512, "layer_2": 128, "output": 10,
          "epochs": 10, "batch_size": 100}
run["parameters"] = params

https://app.neptune.ai/khanh.daoquoc/test-proj/e/TES-17
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## Load data

In [ ]:
# Load data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## Pre-processing

In [ ]:
# Normalize and scale data
scale = 255
train_images = train_images.reshape(60000, 28 * 28)
train_images = train_images.astype('float32') / scale

test_images = test_images.reshape(10000, 28 * 28)
test_images = test_images.astype('float') / scale

# One-hot encode labels
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

## Model

In [ ]:
# Construct the model
model = models.Sequential()
model.add(layers.Dense(params['layer_1'], activation = 'relu', input_shape = (28 * 28,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(params['layer_2'], activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(params['output'], activation = 'softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 512)               401920    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               65664     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                1290      
                                                                 
Total params: 468,874
Trainable params: 468,874
Non-trainable params: 0
_________________________________________________________________


## Compile

In [ ]:
# Compile the model
model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

## Start training

In [ ]:
neptune_cbk = NeptuneCallback(run = run, base_namespace = "training")

model.fit(train_images,
          train_labels,
          epochs = params['epochs'],
          batch_size = params['batch_size'],
          callbacks = [neptune_cbk])

Epoch 1/10
600/600 [==============================] - 9s 13ms/step - loss: 0.2617 - accuracy: 0.9219
Epoch 2/10
600/600 [==============================] - 7s 12ms/step - loss: 0.1111 - accuracy: 0.9666
Epoch 3/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0814 - accuracy: 0.9755
Epoch 4/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0685 - accuracy: 0.9797
Epoch 5/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0573 - accuracy: 0.9837
Epoch 6/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0499 - accuracy: 0.9848
Epoch 7/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0446 - accuracy: 0.9869
Epoch 8/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0396 - accuracy: 0.9882
Epoch 9/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0372 - accuracy: 0.9888
Epoch 10/10
600/600 [==============================] - 7s 12ms/step - loss: 0.0345 - accura

## Evaluate

In [ ]:
eval_metrics = model.evaluate(test_images, test_labels, verbose = 0)

for j, metric in enumerate(eval_metrics):
  run["eval/{}".format(model.metrics_names[j])] = metric

print('Accuracy:', eval_metrics[1])

run.stop()

Accuracy: 0.9840999841690063
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 9 operations to synchronize with Neptune. Do not kill this process.
All 9 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/khanh.daoquoc/test-proj/e/TES-17


## Conclusion

NULL